# 548 hw8 solution

In [ ]:
# numpy = numerical Python, implements arrays (/ matrices)
import numpy as np
# limit number of decimal places printed for floating-point numbers
np.set_printoptions(precision=3)

# scipy = scientific Python, implements operations on arrays / matrices
import scipy as sp
# linalg = linear algebra, implements eigenvalues, matrix inverse, etc
from scipy import linalg as la
# optimize = optimization, root finding, etc
from scipy import optimize as op

# produce matlab-style plots
import matplotlib as mpl
# increase font size on plots
mpl.rc('font',**{'size':18})
# use LaTeX to render symbols
mpl.rc('text',usetex=False)
# animation
from matplotlib import animation as animation
from IPython.display import HTML
mpl.rc('animation', html='jshtml')
# Matlab-style plotting
import matplotlib.pyplot as plt

# symbolic computation, i.e. computer algebra (like Mathematica, Wolfram Alpha)
import sympy as sym


In [ ]:
# pip = Python package manager; "!" means "run at system level"
!pip install slycot
!pip install control

# render SymPy equations nicely in Colaboratory Notebook
def colab_latex_printer(exp,**options):
  from google.colab.output._publish import javascript
  url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
  javascript(url=url)
  return sym.printing.latex(exp,**options)

sym.init_printing(use_latex="mathjax",latex_printer=colab_latex_printer)

In [ ]:
# Python's Control Systems Toolbox
import control as ctrl

# SciPy module that implements many of the routines in ctrl
from scipy import signal as sig

# gain margin of an LQG regulator

***Purpose:*** verify that the gain margin of an LQG regulator can be arbitrarily small, so combining the linear-quadratic controller with the Kalman filter observer does not necessarily yield a robust closed-loop system.

Consider the following example from [Doyle's 1978 paper](https://paperpile.com/shared/BRx9bl):

$$
\dot{x}
=
\left[ \begin{array}{c} 1 & 1 \\ 0 & 1 \end{array} \right]
x
+
\left[ \begin{array}{c} 0 \\ 1 \end{array} \right]
u
+
\left[ \begin{array}{c} 1 \\ 1 \end{array} \right]
w,
$$

$$
y = 
\left[ \begin{array}{c} 1 & 0 \end{array} \right] x + v,
$$

where:
$x\in\mathbb{R}^2$ is the state, 
$u\in\mathbb{R}$ is the control input, 
$w\in\mathbb{R}$ is the input disturbance, 
and
$v\in\mathbb{R}$ is the output disturbance.

Assume $w$ and $v$ are independent, zero-mean, Gaussian (*), with variances $\operatorname{E}[w w^\top] = \sigma^2 > 0$ and $\operatorname{E}[v v^\top] = 1$.

(*) More formally, we should refer to these as [Wiener processes](https://en.wikipedia.org/wiki/Wiener_process), since they are defined in continuous-time.

We seek to minimize the expected infinite-horizon linear-quadratic error 

$$
\operatorname{E}\left[ \int_0^\infty x^\top Q x + u^\top R u \right],\
Q = q \left[ \begin{array}{c} 1 \\ 1 \end{array} \right] \left[ \begin{array}{c} 1 & 1 \end{array} \right]
,\ 
q > 0
,\
R = 1.
$$

***Note:*** there are two free parameters in the model:  $\sigma^2$ and $q$.  You can choose whether to solve the following problems analytically (by hand or using a computer algebra system like `SymPy`) or numerically (by verifying the conclusions for different numerical values of $\omega > 0$ and $q > 0$ -- ideally, many randomly-selected values of these parameters).



(a) *Verify that the gain matrices that define the linear-quadratic controller (LQ) and Kalman filter observer (KF) are the transposes of one another when $q = \sigma^2$.*

***Hint:*** use duality of the infinite-horizon continuous-time LQ and KF problems:
* numerically, use the fact that `K = LQR(A,B,Q,R)` and `L = LQR(A.T,C.T,W,V).T`  where $W = \operatorname{E}[w w^\top] \left[ \begin{array}{c} 1 \\ 1 \end{array} \right] \left[ \begin{array}{c} 1 & 1 \end{array} \right]$ and $V =  \operatorname{E}[v v^\top] = 1$;
* analytically, use the fact that $K = R^{-1} B^\top P$ and $L = S C V^{-1}$ where $P$ and $S$ satisfy the *algebraic Riccati equations*
$$0 = A^\top P + P A - P B R^{-1} B^\top P + Q, $$ 
$$0 = A S + S A^\top - S C^\top V^{-1} C S + W. $$ 


**Solution:**

We want to verify that $K = L^T$. Numerically, we have that:

In [ ]:
# System matrices
A = np.array([[1,1],[0,1]])
B = np.array([[0],[1]])
C = np.array([[1,0]])

R = np.array([[1]])
V = np.array([[1]])

Q = lambda q: q * np.dot(np.array([[1],[1]]),np.array([[1,1]])) 
W = lambda w: w * np.dot(np.array([[1],[1]]),np.array([[1,1]]))

In [ ]:
# Verify claim with random values of sigma^2
sig2 = np.random.normal(size=100)
verify_K_LT = []
for _, s in enumerate(sig2):
  K,P,ELQ = ctrl.lqr(A,B,Q(s),R)
  assert np.allclose(0,np.dot(A.T,P)+np.dot(P,A)-np.dot( P, np.dot(B, np.dot( np.linalg.inv(R), np.dot(B.T,P)))) + Q(s))
  L,S,EKF = ctrl.lqr(A.T,C.T,W(s),V)
  assert np.allclose(0,np.dot(A,S)+np.dot(S,A.T)-np.dot( S, np.dot(C.T, np.dot( np.linalg.inv(V), np.dot(C,S)))) + W(s))
  # verify equality
  verify_K_LT.append(np.allclose(K, L.T))

# Verify if K = LT for all cases
np.all(verify_K_LT)

True


(b) *Verify that the LQ controller is $u = - K x = -(2 + \sqrt{4 + q})\left[ \begin{array}{c} 1 & 1 \end{array} \right] x$.*

***Note:*** combining (a.) and (b.), we conclude that the Kalman filter's gain matrix is $L^\top = (2 + \sqrt{4 + \frac{1}{\omega}})\left[ \begin{array}{c} 1 & 1 \end{array} \right]$.)


**Solution:**

Verifying numerically:

In [ ]:
# Verify claim with random values of sigma^2
q_rand = np.random.normal(size=100)
verify_K_Keval = []
for _, q in enumerate(q_rand):
  K,_,_ = ctrl.lqr(A,B,Q(q),R)
  K_eval = np.dot(2 + np.sqrt(4 + q), np.array([[1,1]]))
  # verify equality
  verify_K_Keval.append(np.allclose(K, K_eval))

# Verify if K = LT for all cases
np.all(verify_K_Keval)

True


(c) *Verify that closing the loop using the LQ controller and KF observer yields*

$$
\left[ \begin{array}{c} \dot{x} \\ \dot{\widehat{x}} \end{array} \right]
=
A
\left[ \begin{array}{c} x \\ \widehat{x} \end{array} \right]
+
E w
=
\left[ \begin{array}{c} 1 & 1 & 0 & 0 \\ 0 & 1 & -f & -f \\ d & 0 & 1-d & 1 \\ d & 0 & -d-f & 1-f \end{array} \right]
\left[ \begin{array}{c} x \\ \widehat{x} \end{array} \right]
+
E w
$$

*where $f = 2 + \sqrt{4 + q}$, $d = 2 + \sqrt{4 + \frac{1}{\omega}}$.*


***Hint:*** you need to verify the given expression for $A$; you do not need to determine $E$ (though it is straightforward to do so).


**Solution:**

Ignoring the noise component of the given dynamics, the LQ feedback control given by $u = -K\hat{x}$ results in the dynamics:

$$\dot{x} = Ax + Bu = Ax - BK\hat{x}$$

where $A = \left[ \begin{array}{c} 1 & 1 \\ 0 & 1 \end{array} \right]$ and $-BK = \left[ \begin{array}{c} 0 \\ 1 \end{array} \right] f\left[ \begin{array}{c} 1 , 1 \end{array} \right] = \left[ \begin{array}{c} 0 & 0 \\ -f & -f \end{array} \right]$ 

The LQG is given by:

$$\dot{\hat{x}} = A\hat{x} + Bu + L(y-\hat{y})$$
$$\qquad = A\hat{x} - BK\hat{x} + LC(x-\hat{x})$$
$$\qquad = (A - BK - LC)\hat{x} + LCx$$

where
$LC = d\left[ \begin{array}{c} 1 , 1 \end{array} \right]^T \left[ \begin{array}{c} 1 , 0 \end{array} \right] = 
\left[ \begin{array}{c} d & 0 \\ d & 0 \end{array} \right]$ and

$A - BK - LC = \left[ \begin{array}{c} 1 & 1 \\ 0 & 1 \end{array} \right] + \left[ \begin{array}{c} 0 & 0 \\ -f & -f \end{array} \right] - \left[ \begin{array}{c} d & 0 \\ d & 0 \end{array} \right] = \left[ \begin{array}{c} 1-d & 1 \\ -d-f & 1-f \end{array} \right]$

putting $\dot{x}$ and $\dot{\hat{x}}$ results in:

$$
\left[ \begin{array}{c} \dot{x} \\ \dot{\widehat{x}} \end{array} \right]
=
\left[ \begin{array}{c} 1 & 1 & 0 & 0 \\ 0 & 1 & -f & -f \\ d & 0 & 1-d & 1 \\ d & 0 & -d-f & 1-f \end{array} \right]
\left[ \begin{array}{c} x \\ \widehat{x} \end{array} \right]
+
E w
$$
where the matrix $E$ is the noise input matrix for the closed loo[ system.


Suppose that the resulting closed-loop controller has a scalar gain $m$ (nominally unity) associated with the input matrix, but that only the nominal value of this gain is known to the filter. The full systcm matrix then
becomes 

$$
\left[ \begin{array}{c} \dot{x} \\ \dot{\widehat{x}} \end{array} \right]
=
A
\left[ \begin{array}{c} x \\ \widehat{x} \end{array} \right]
+
E w
=
\left[ \begin{array}{c} 1 & 1 & 0 & 0 \\ 0 & 1 & -mf & -mf \\ d & 0 & 1-d & 1 \\ d & 0 & -d-f & 1-f \end{array} \right]
\left[ \begin{array}{c} x \\ \widehat{x} \end{array} \right]
+
E w.
$$

***Note:*** the inclusion of parameter $m$ can be used to assess the *gain margin* of the closed-loop LQG = LQ + KF controller.


(d) *Verify that, as $q$ or $1/\omega$ get larger, a smaller value of $m$ is needed to make the closed-loop system unstable.*

***Hint:*** the original paper verifies this fact by applying Routh-Hurwitz stability criteria to the characteristic polynomial of $A$, above -- you are welcome to reproduce this argument, or to numerically evaluate the eigenvalues of $A$ for different choices of $q$, $1/\omega$, and $m$. 

**Solution:**

Verifying numerically,

In [ ]:
f = lambda q: 2 + np.sqrt(4 + q)
d = lambda q: 2 + np.sqrt(4 + q) # q = 1/omega

m_span = np.linspace(0.9, 1.1, 10) # perturb m = 1 by plus/minus 0.1
q_span = np.linspace(1, 100, 10)
A_stable = np.zeros((10, 10)) 
for idx_1, m in enumerate(m_span):
  for idx_2, q in enumerate(q_span):
    A = np.array([[1, 1, 0, 0],[0, 1, -m*f(q), -m*f(q)],[d(q), 0, 1-d(q), 1],[d(q), 0, -d(q)-f(q), 1-f(q)]])
    max_eig = np.max(np.real(la.eigvals(A)))
    A_stable[idx_1, idx_2] = 0 if max_eig > 0 else 1

print(f'Range of m = {m_span}') # rows of A_stable matrix
print(f'Range of q = {q_span}')# columns of A_stable matrix
# print(f'Is system stable for given m and q = {A_stable}') 
A_stable # 0 if unstable and 1 if stable

Range of m = [0.9   0.922 0.944 0.967 0.989 1.011 1.033 1.056 1.078 1.1  ]
Range of q = [  1.  12.  23.  34.  45.  56.  67.  78.  89. 100.]


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

It can be observed from the array above that as q increases and m is pertubed by a small amount (in this case $m = 1 \pm 0.1$) the system goes unstable.

The following conclusions are from [Doyle's 1978 paper](https://paperpile.com/shared/BRx9bl):

> It is interesting to note that the margins deteriorate as control weight
gets small and/or system driving noise gets large. In modem control
folklore, these have often been considered ad hoc means of improving
sensitivity.

> It is also important to recognize that vanishing margins are not only
associated with open-loop unstable systems. It is easy to construct
minimum phase, open-loop stable counterexamples for which the
margins are arbitrarily small.

> The point of these examples is that LQG solutions, unlike LQ solutions, provide no global system-independent guaranteed robustness properties. Like their more classical colleagues, modern LQG designers
are obliged to test their margins for each speafic design.

> It may, however, be possible to improve the robustness of a given
design by relaxing the optimality of the filter with respect to error
properties. A promising approach appears to be the introduction of
certain fictitiou system noises in the filter design procedure. This
approach will be the topic of future papers.

# linear analysis

***Purpose:*** work with induced norms of linear operators -- although it may initially seem abstract,  we will use this mathematical construction for the very practical problem of analyzing and synthesizing robustness in control systems.

Given a linear function $L : V \rightarrow W$ between vector spaces $V,\ W$ with corresponding norms $\| \cdot \|_V,\ \| \cdot \|_W$, recall that we define the *induced norm* $\| L \|_{V\rightarrow W}$ to be the largest "gain" or "amplification" of a vector $v\in V$ through operator $L$:

$$
\| L \|_{V\rightarrow W} = \max_{v \neq 0} \frac{\| L(v) \|_W}{\| v \|_V}.
$$

Since any matrix $M\in\mathbb{R}^{m\times n}$ defines a linear operator $L_M : \mathbb{R}^n \rightarrow \mathbb{R}^m$ via matrix multiplication, $L_M(v) = M v$, we can start by assessing induced norms of matrices for different choices of norm on the domain $\mathbb{R}^n$ and codomain $\mathbb{R}^m$.



(a) *Verify that $\| M \| = \max_{1 \leq j \leq n} \sum_{i = 1}^m | M_{ij} |$ when the $1$-norm is used on $\mathbb{R}^n$ and $\mathbb{R}^m$; this formula can be summarized as evaluating the "maximum absolute column sum".*

**Solution:**

We want to verify that $\| M \|_1 = \max_{1 \leq j \leq n} \sum_{i = 1}^m | M_{ij} |$. The induced matrix $1$-norm is 
$$
\| M \|_1 = \max_{v \neq 0} \frac{\| M v \|_1}{\| v \|_1}
$$

and normalizing $\| v \|_1 = 1$, results in : 
$$ \| M \|_1 = \max_{\| v \|_1 = 1} \| M v \|_1$$

Given that the $1$-norm of a vector $v$ is $\| v \|_1 = \sum_{j = 1}^n | v_j |$, we have that:
$$\| M v \|_1 = \sum_{i = 1}^m |(Mv)_i| \leq \sum_{i = 1}^m \sum_{j = 1}^n  |m_{ij}| |v_j|$$

$$ \qquad \qquad \qquad \qquad =\sum_{j = 1}^n \big( |v_j| \sum_{i = 1}^m |m_{ij}| \big)  \leq \| v \|_1 \Big(\max_{1 \leq j \leq n} \sum_{i = 1}^m |m_{ij}|\Big) $$

The first inequality is obtained by applyting the triangle inequality property of norms, while the second is from the Holder's inequality. Since $\| v \|_1 = 1$, we have that:

$$\| M v \|_1 \leq \max_{1 \leq j \leq n} \sum_{i = 1}^m |m_{ij}|$$

To show the above inequality holds with equality, let the $k^{th}$ column of $M$ have the maximum absolute sum. Since we normalized $\| v \|_1 = 1$, we can choose $v$ to be a standard basis vector where the $k^{th}$ component is a unit vector, that is, $v = e_k =[0, \dots, 1, \dots, 0]^T$. Then:

$$\| M v \|_1 =  \sum_{i = 1}^m \Bigg| \sum_{j = 1}^n m_{ij}v_j \Bigg| = \sum_{i = 1}^m |m_{ik}| = \max_{1 \leq j \leq n} \sum_{i = 1}^m |m_{ij}|$$

We therefore have that:

$$ \| M \|_1 = \max_{\| v \|_1 = 1} \| M v \|_1 = \max_{1 \leq j \leq n} \sum_{i = 1}^m |m_{ij}|$$






(b) *Verify that $\| M \| = \max_{1 \leq i \leq m} \sum_{j = 1}^n | M_{ij} |$ when the $\infty$-norm is used on $\mathbb{R}^n$ and $\mathbb{R}^m$; this formula can be summarized as evaluating the "maximum absolute row sum".*

**Solution:**

We want to verify that $\| M \|_\infty = \max_{1 \leq i \leq m} \sum_{j = 1}^n | M_{ij} |$. The induced matrix $\infty$-norm is 
$$
\| M \|_\infty = \max_{v \neq 0} \frac{\| M v \|_\infty}{\| v \|_\infty}
$$

and normalizing $\| v \|_\infty = 1$, results in: 
$$\| M \|_\infty = \max_{\| v \|_\infty = 1} \| M v \|_\infty$$

Given that the $\infty$-norm of a vector $v$ is $\| v \|_\infty = \max_{j} | v_j |$, we have that:
$$\| M v \|_\infty = \max_{i} |(Mv)_i| \leq \max_{i} \Big(\sum_{j = 1}^n  |m_{ij}v_j|\Big) \leq \max_{i} \| v \|_\infty \Big(\sum_{j = 1}^n  |m_{ij}| \Big)$$

The first inequality is obtained by applyting the triangle inequality property of norms, while the second is from the Holder's inequality. Since $\| v \|_\infty = 1$, we have that:

$$\| M v \|_\infty \leq \max_{i} \sum_{j = 1}^n  |m_{ij}| $$

Similar to the steps taken in (a), we show the above inequality holds with equality, by taking the $k^{th}$ column of $M$ that has the maximum absolute sum and $v$ to be a standard basis vector where the $k^{th}$ component is a unit vector, resulting in:

$$ \| M \|_\infty = \max_{\| v \|_\infty = 1} \| M v \|_\infty = \max_{1 \leq i \leq m} \sum_{j = 1}^n | M_{ij} |$$

(c) *Verify that $\| M \| = \sigma_{\max}(M) = \lambda_{\max}\left(M^\top M\right)^{1/2}$ when the $2$-norm is used on $\mathbb{R}^n$ and $\mathbb{R}^m$; this formula can be summarized as evaluating the "maximum singular value".*

***Hint:*** you may find it convenient to use the [singular value decomposition (SVD)](https://en.wikipedia.org/wiki/Singular_value_decomposition) of the matrix $M$ to verify (a.--c.).

**Solution:**

We want to verify that $\| M \|_2 = \sigma_{\max}(M) = \lambda_{\max}\left(M^\top M\right)^{1/2}$. The induced matrix $2$-norm is 
$$
\| M \|_2 = \max_{v \neq 0} \frac{\| M v \|_2}{\| v \|_2}
$$

and normalizing $\| v \|_2 = 1$, results in: 
$$\| M \|_2 = \max_{\| v \|_2 = 1} \| M v \|_2$$

Taking the SVD of $M$ gives $M = U \Sigma V^T$ where $U$ and $V$ are unitary matrices, then  
$$ \| M v \|_2 =  \| U \Sigma V^T v \|_2 = \|\Sigma V^T v \|_2$$

where the last equality holds since vectors are invariant under unitary matrices multiplication. Given that the $2$-norm of any vector $y$ is $\| y \|_2 = \Big(\sum_{j = 1}^n | y_j |^2\Big)^{\frac{1}{2}}$ and since $\Sigma = diag(\sigma_1, \dots, \sigma_n)$ arranged in decreasing order such that $\sigma_1$ is the largest singular value. Let $y = V^T v$:

$$ \| M v \|_2 = \|\Sigma y \|_2 = \Big(\sum_{j = 1}^n | \sigma_j y_j |^2\Big)^{\frac{1}{2}}$$

Since the matrix $V$ is also a unitary matrix, $\| y \|_2 = \| v \|_2 = 1$. The maximum of the above equation is obtained when the $y$ is chosen to be a standard basis vector $e_1$. Therefore

$$\| M \|_2 = \max_{\| y \|_2 = 1} \|\Sigma y \|_2 = \sigma_{\max}(M)$$



For a more complex example that may be familiar from an undergraduate course on signals or structures, we consider the [convolution](https://en.wikipedia.org/wiki/Convolution) operation.  With $f:\mathbb{R}\rightarrow \mathbb{R}$ denoting the *convolution kernel*, and assuming $f\in L_1$ (i.e. $\| f \|_1 = \int_{-\infty}^{+\infty} |f(\tau)| d\tau < \infty$), we define an operator $F:L_\infty \rightarrow L_\infty$ applicable to any $u\in L_\infty$ by

$$
(F u)(t) = (f \ast u)(t) = \int_{-\infty}^{+\infty} f(t - \tau) u(\tau) d\tau.
$$

(d) Verify that $F$ is linear, i.e. that $F(\alpha u_1 + u_2) = \alpha F u_1 + F u_2$ for all $\alpha\in\mathbb{R}$ and $u_1,u_2\in L_\infty$.

**Solution:**

Substituting $F(\alpha u_1 + u_2)$ into the definition above:
$$
\Big(F (\alpha u_1 + u_2)\Big)(t) = \int_{-\infty}^{+\infty} f(t - \tau) (\alpha u_1 + u_2)(\tau) d\tau
$$

$$
\qquad \qquad \qquad \qquad \qquad \qquad \qquad \ = \alpha  \int_{-\infty}^{+\infty} f(t - \tau) u_1 (\tau) d\tau + \int_{-\infty}^{+\infty}f(t - \tau) + u_2(\tau) d\tau
$$

$$
 = \alpha F u_1 + F u_2
$$


(e) Verify that the induced norm $\| F \|_{L_\infty\rightarrow L_\infty}$ is bounded above by $\| f \|_1$.  

**Solution:**

From the $\infty$-norm result in (b) and normalizing u such that $\| u \|_\infty = 1$, we have:
$$\| F \|_\infty = \max_{\| u \|_\infty = 1} \| F u \|_\infty \leq  \|u \|_\infty \Bigg(\max_\tau \int_{-\infty}^{\infty}  |f(t-\tau)| d \tau\Bigg)$$

$$ = \|u \|_\infty \|f \|_1$$

***Bonus:*** show that, in fact, $\| F \|_{L_\infty\rightarrow L_\infty} = \| f \|_1$.

***Interesting connection:*** the Fourier transform of signal $u$ is defined by evaluating the convolution with the kernel $e^{-j\omega t}$ at $t = 0$.